In [1]:
%matplotlib inline  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
climate = pd.read_csv('climate.csv', sep='\t')
deliveries = pd.read_csv('deliveries.csv', sep='\t')

In [3]:
climate['crushdate'] = pd.to_datetime(climate['day'], format='%Y-%m-%d %H:%M:%S')
deliveries['crushdate'] = pd.to_datetime(deliveries['crushdate'], format='%m/%d/%y')

climate['year'] = climate['crushdate'].apply(lambda x: x.year).astype(int)
climate['month'] = climate['crushdate'].apply(lambda x: x.month).astype(int)
climate['day'] = climate['crushdate'].apply(lambda x: x.day).astype(int)

deliveries['year'] = deliveries['crushdate'].apply(lambda x: x.year).astype(int)
deliveries['month'] = deliveries['crushdate'].apply(lambda x: x.month).astype(int)
deliveries['day'] = deliveries['crushdate'].apply(lambda x: x.day).astype(int)

In [4]:
deliveries['tier'] = deliveries['programno'].apply(lambda x: x[:3])
deliveries['varietal'] = deliveries['programno'].apply(lambda x: x[3:6]).astype(int)
deliveries['origin'] = deliveries['programno'].apply(lambda x: x[6:])

In [5]:
# Look at one varietal first
v = deliveries[deliveries['varietal'] == 101]

In [6]:
v = v.sort_values(['crushdate', 'pblk'])
avg = v.groupby(['crushdate', 'pblk']).agg({'brix':np.mean, 'tons':np.mean})

v = v.drop(['brix', 'tons'], axis=1)
v = v.drop_duplicates(['crushdate', 'pblk'])

v['avg_brix'] = avg['brix'].tolist()
v['avg_tons'] = avg['tons'].tolist()

In [7]:
print(v.shape)
v.head()

(6103, 14)


,weighttagno,tagyear,siteno,programno,crushdate,pblk,year,month,day,tier,varietal,origin,avg_brix,avg_tons
66,793682,1,8,TLV101CAL,2001-08-11,100094,2001,8,11,TLV,101,CAL,22.900000,1.512525
78,793794,1,8,TLV101CAL,2001-08-11,101177,2001,8,11,TLV,101,CAL,23.200000,1.494967
64,793642,1,8,EJG101CAL,2001-08-11,101545,2001,8,11,EJG,101,CAL,22.571429,1.762500
67,793683,1,8,EJG101CAL,2001-08-11,101669,2001,8,11,EJG,101,CAL,21.675000,1.553650
79,793828,1,8,TLV101CAL,2001-08-11,102031,2001,8,11,TLV,101,CAL,23.550000,1.664950


In [8]:
columns = ['crushdate', 'year', 'month', 'day', 'pblk', 'avg_brix', 'tier', 'origin']
v = v[columns]

In [9]:
v['pblk'].describe()

count      6103.00000
mean     112988.32099
std       11885.28644
min      100094.00000
25%      102499.00000
50%      111315.00000
75%      120877.00000
max      141153.00000
Name: pblk, dtype: float64

In [10]:
# Add more columns to the dataframe using the climate data
# Columns to be added:
# logitude, latitude of the pblk
# ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp of yesterday
# average ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for the past one week
# average ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for the past one month
# average ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for the past three month
# count of ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp that is greater than the mean for the past one week
# count of ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp that is greater than the mean for the past one month
# count of ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp that is greater than the mean for the past three month
# low rank approximation of ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for each pblk

In [11]:
# Add longitude, latitude
location = climate[['pblk', 'long', 'lat']]
location = location.drop_duplicates(['pblk', 'long', 'lat'])
v = pd.merge(v, location, how="left", on='pblk')
v.head()

,crushdate,year,month,day,pblk,avg_brix,tier,origin,long,lat
0,2001-08-11,2001,8,11,100094,22.900000,TLV,CAL,-8493.8518,2666.9468
1,2001-08-11,2001,8,11,101177,23.200000,TLV,CAL,-8493.5375,2669.7853
2,2001-08-11,2001,8,11,101545,22.571429,EJG,CAL,-8472.4521,2634.5172
3,2001-08-11,2001,8,11,101669,21.675000,EJG,CAL,-8454.9913,2617.1782
4,2001-08-11,2001,8,11,102031,23.550000,TLV,CAL,-8483.9258,2667.5747


In [12]:
# Add ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp of yesterday
features = ['ddays50', 'ddays50_97', 'ddays97', 'humidity', 'cloud_cover', 'pressure', 'maxtemp', 'mintemp']
yesterday_data = pd.DataFrame(columns=features)

for i in range(0, v.shape[0]):
    pblk = v.iloc[i].pblk
    yesterday = v.iloc[i].crushdate - timedelta(days=1)
    
    yesterday_data.loc[i] = climate[(climate['pblk'] == pblk) & 
                                    (climate['year'] == yesterday.year) & 
                                    (climate['month'] == yesterday.month) & 
                                    (climate['day'] == yesterday.day)][features].iloc[0].tolist()

In [13]:
for feature in features:
    v['yesterday_' + feature] = yesterday_data[feature]

In [14]:
v.head()

,crushdate,year,month,day,pblk,avg_brix,tier,origin,long,lat,yesterday_ddays50,yesterday_ddays50_97,yesterday_ddays97,yesterday_humidity,yesterday_cloud_cover,yesterday_pressure,yesterday_maxtemp,yesterday_mintemp
0,2001-08-11,2001,8,11,100094,22.900000,TLV,CAL,-8493.8518,2666.9468,20.11,20.11,0.0,0.59,0.08,1015.50,88.76,59.70
1,2001-08-11,2001,8,11,101177,23.200000,TLV,CAL,-8493.5375,2669.7853,20.81,20.81,0.0,0.56,0.05,1015.27,89.74,59.65
2,2001-08-11,2001,8,11,101545,22.571429,EJG,CAL,-8472.4521,2634.5172,22.42,22.42,0.0,0.58,0.03,1015.03,90.59,59.24
3,2001-08-11,2001,8,11,101669,21.675000,EJG,CAL,-8454.9913,2617.1782,23.55,23.55,0.0,0.56,0.02,1014.72,91.17,60.03
4,2001-08-11,2001,8,11,102031,23.550000,TLV,CAL,-8483.9258,2667.5747,21.44,21.44,0.0,0.57,0.03,1015.19,90.64,59.43


In [15]:
# Add average ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for the past one week
week_data = pd.DataFrame(columns=features)

for i in range(0, v.shape[0]):
    pblk = v.iloc[i].pblk
    yesterday = v.iloc[i].crushdate - timedelta(days=1)
    last_week = v.iloc[i].crushdate - timedelta(days=7)
    
    week_data.loc[i] = climate[(climate['pblk'] == pblk) & 
                               (climate['crushdate'] >= last_week) & 
                               (climate['crushdate'] <= yesterday)][features].mean().tolist()

In [16]:
week_data.head()

,ddays50,ddays50_97,ddays97,humidity,cloud_cover,pressure,maxtemp,mintemp
0,25.221667,22.835000,0.101667,0.536667,0.055000,1013.343333,94.176667,61.663333
1,25.848333,23.446667,0.115000,0.525000,0.055000,1013.168333,94.736667,62.040000
2,28.293333,24.498333,0.203333,0.480000,0.013333,1012.951667,96.156667,63.788333
3,28.626667,24.831667,0.205000,0.491667,0.011667,1012.763333,97.100000,63.491667
4,26.783333,23.356667,0.160000,0.506667,0.036667,1013.088333,95.550000,62.575000


In [17]:
for feature in features:
    v['week_' + feature] = week_data[feature]

In [18]:
month_data = pd.DataFrame(columns=features)

for i in range(0, v.shape[0]):
    pblk = v.iloc[i].pblk
    yesterday = v.iloc[i].crushdate - timedelta(days=1)
    last_month = v.iloc[i].crushdate - timedelta(days=31)
    
    month_data.loc[i] = climate[(climate['pblk'] == pblk) & 
                                (climate['crushdate'] >= last_month) & 
                                (climate['crushdate'] <= yesterday)][features].mean().tolist()

In [19]:
month_data.head()

,ddays50,ddays50_97,ddays97,humidity,cloud_cover,pressure,maxtemp,mintemp
0,20.262000,19.784667,0.020333,0.590333,0.075000,1012.684000,88.369000,57.618333
1,20.926667,20.446333,0.023000,0.571667,0.052667,1012.468667,89.258000,57.788333
2,23.600667,22.841667,0.040667,0.520333,0.024333,1012.310000,90.771333,59.541333
3,24.221333,23.462333,0.041000,0.522333,0.014667,1012.104667,91.608667,59.480000
4,21.932000,21.246667,0.032000,0.552000,0.036333,1012.398667,90.130667,58.305333


In [20]:
for feature in features:
    v['month_' + feature] = month_data[feature]

In [21]:
month3_data = pd.DataFrame(columns=features)

for i in range(0, v.shape[0]):
    pblk = v.iloc[i].pblk
    yesterday = v.iloc[i].crushdate - timedelta(days=1)
    last_month3 = v.iloc[i].crushdate - timedelta(days=93)
    
    month3_data.loc[i] = climate[(climate['pblk'] == pblk) & 
                               (climate['crushdate'] >= last_month3) & 
                               (climate['crushdate'] <= yesterday)][features].mean().tolist()

In [22]:
month3_data.head()

,ddays50,ddays50_97,ddays97,humidity,cloud_cover,pressure,maxtemp,mintemp
0,22.128587,20.857174,0.079674,0.521087,0.081957,1012.660435,89.704457,57.971304
1,22.729457,21.344348,0.086413,0.505978,0.080000,1012.482174,90.379565,58.225543
2,25.257826,23.734348,0.097391,0.450870,0.026739,1012.268152,91.959130,60.715978
3,25.572826,24.009022,0.095217,0.466087,0.024239,1012.078587,92.524022,60.443804
4,23.676304,22.026630,0.095761,0.484783,0.056739,1012.401087,91.176196,58.982935


In [23]:
for feature in features:
    v['month3_' + feature] = month3_data[feature]

In [55]:
v.to_csv('v.csv', index=False)

In [ ]:
v = pd.read_csv('v.csv')

In [56]:
lr_ddays50 = pd.read_csv('lr_ddays50.csv')
lr_humidity = pd.read_csv('lr_humidity.csv')
lr_cloud_cover = pd.read_csv('lr_cloud_cover.csv')
lr_maxtemp = pd.read_csv('lr_maxtemp.csv')

lr_ddays50.head()

,ddays50_1,ddays50_2,ddays50_3,pblk
0,1611.355802,-12.218990,0.634748,100013
1,1367.716021,12.208003,-1.045634,100015
2,1370.836849,12.068548,-1.053069,100024
3,1727.012415,-16.562810,0.041056,100055
4,1346.071443,14.417660,-0.434827,100089


In [57]:
v = pd.merge(v, lr_ddays50, how="left", on='pblk')
v = pd.merge(v, lr_humidity, how="left", on='pblk')
v = pd.merge(v, lr_cloud_cover, how="left", on='pblk')
v = pd.merge(v, lr_maxtemp, how="left", on='pblk')

In [61]:
v.head()

,crushdate,pblk,avg_brix,long,lat,yesterday_ddays50,yesterday_ddays50_97,yesterday_ddays97,yesterday_humidity,yesterday_cloud_cover,...,ddays50_3_y,humidity_1_y,humidity_2_y,humidity_3_y,cloud_cover_1_y,cloud_cover_2_y,cloud_cover_3_y,maxtemp_1_y,maxtemp_2_y,maxtemp_3_y
0,2001-08-11,100094,-0.438671,-0.609619,0.507389,-0.644272,-0.495267,-0.323883,0.906516,0.553765,...,-0.880299,5.165003,58.323101,0.746172,53.801215,-17.179744,-16.226594,6636.494152,8.161382,0.982966
1,2001-08-11,101177,-0.250283,-0.601403,0.574836,-0.517227,-0.324215,-0.323883,0.597813,0.135960,...,-0.438163,5.166123,58.343320,0.722323,54.070873,-17.434609,-16.148007,6644.420353,8.387873,0.768754
2,2001-08-11,101545,-0.645001,-0.050168,-0.263187,-0.225024,0.069205,-0.323883,0.803615,-0.142577,...,-2.855970,4.854513,55.531924,-0.386679,63.004059,28.060896,10.997136,6776.373837,2.546416,-0.046823
3,2001-08-11,101669,-1.207922,0.406310,-0.675187,-0.019937,0.345332,-0.323883,0.597813,-0.281846,...,-1.393013,4.950029,56.548072,-0.549698,63.025726,27.942414,11.635398,6784.914007,1.323419,-1.048153
4,2001-08-11,102031,-0.030496,-0.350124,0.522309,-0.402887,-0.170268,-0.323883,0.700714,-0.142577,...,-1.450714,5.067358,57.442546,0.318090,50.110882,-14.240416,-10.846082,6714.483520,7.168350,-0.065554


In [59]:
# Standardize continuous columns and create dummy for categorical
continuous = ['long', 'lat', 'yesterday_ddays50', 'yesterday_ddays50_97', 'yesterday_ddays97',
              'yesterday_humidity', 'yesterday_cloud_cover', 'yesterday_pressure', 'yesterday_maxtemp',
              'yesterday_mintemp', 'week_ddays50', 'week_ddays50_97', 'week_ddays97', 'week_humidity',
              'week_cloud_cover', 'week_pressure', 'week_maxtemp', 'week_mintemp', 'month_ddays50',
              'month_ddays50_97', 'month_ddays97', 'month_humidity', 'month_cloud_cover', 'month_pressure',
              'month_maxtemp', 'month_mintemp', 'month3_ddays50', 'month3_ddays50_97', 'month3_ddays97',
              'month3_humidity', 'month3_cloud_cover', 'month3_pressure', 'month3_maxtemp', 'month3_mintemp',
              'ddays50_1', 'ddays50_2', 'ddays50_3', 'humidity_1', 'humidity_2', 'humidity_3', 'cloud_cover_1',
              'cloud_cover_2', 'cloud_cover_3', 'maxtemp_1', 'maxtemp_2', 'maxtemp_3']
categorical = ['year', 'month', 'day', 'tier', 'origin']

for item in categorical:
    dummy = pd.get_dummies(v[item], prefix=item)
    v = v.join(dummy.ix[:, :])
    v = v.drop(item, axis=1)

KeyError: 'year'

In [34]:
from sklearn import preprocessing

scalar = preprocessing.StandardScaler()
scalar.fit(v[continuous])
v[continuous] = scalar.transform(v[continuous])

In [36]:
index = v[['crushdate', 'pblk']]
y = v['avg_brix']
X = v.drop(['crushdate', 'pblk', 'avg_brix'], axis=1)

In [82]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.01)
model.fit(X, y)
print('R^2 score: ', model.score(X, y))

R^2 score:  0.638232606394


In [83]:
coef = pd.DataFrame(columns=['feature', 'coef'])
coef['feature'] = X.columns.tolist()
coef['coef'] = model.coef_

coef = coef.sort_values('coef')
coef.head(30)

,feature,coef
97,tier_CHP,-1.275522
113,origin_CAL,-0.373602
50,year_2005,-0.360486
42,cloud_cover_3,-0.243600
19,month_ddays50_97,-0.192685
28,month3_ddays97,-0.146443
112,tier_VAL,-0.112136
5,yesterday_humidity,-0.080860
48,year_2003,-0.052024
13,week_humidity,-0.033603


In [74]:
import statsmodels.api as sm

results = sm.OLS(y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               avg_brix   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     119.9
Date:                Sun, 22 Jan 2017   Prob (F-statistic):               0.00
Time:                        15:10:19   Log-Likelihood:                -4957.3
No. Observations:                6103   AIC:                         1.015e+04
Df Residuals:                    5984   BIC:                         1.095e+04
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
long                     -0.2290      0.041     -5.568      0.000        -0.310    -0.148
lat                      -0.1453      0.045     -3.235      0.001        -0.233    -0.057
yesterday_ddays50         0.0430      0.052      0.827      0.408        -0.059     0.145
yesterday_ddays50_97     -0.0688      0.024     -2.917      0.004        -0.115    -0.023
yesterday_ddays97        -0.0364      0.017     -2.157      0.031        -0.069    -0.003
yesterday_humidity       -0.0781      0.014     -5.496      0.000        -0.106    -0.050
yesterday_cloud_cover    -0.0186      0.010     -1.883      0.060        -0.038     0.001
yesterday_pressure        0.0062      0.009      0.665      0.506        -0.012     0.024
yesterday_maxtemp         0.0402      0.028      1.431      0.152        -0.015     0.095
yesterday_mintemp         0.0419      0.023      1.832      0.067        -0.003     0.087
week_ddays50             -0.1258      0.108     -1.167      0.243        -0.337     0.085
week_ddays50_97          -0.0888      0.036     -2.450      0.014        -0.160    -0.018
week_ddays97             -0.0582      0.024     -2.474      0.013        -0.104    -0.012
week_humidity            -0.0838      0.020     -4.105      0.000        -0.124    -0.044
week_cloud_cover          0.0042      0.012      0.364      0.716        -0.018     0.027
week_pressure            -0.0342      0.013     -2.592      0.010        -0.060    -0.008
week_maxtemp              0.1943      0.054      3.605      0.000         0.089     0.300
week_mintemp              0.0918      0.051      1.803      0.071        -0.008     0.192
month_ddays50            -0.6743      0.200     -3.371      0.001        -1.066    -0.282
month_ddays50_97          0.1664      0.061      2.719      0.007         0.046     0.286
month_ddays97             0.1141      0.029      3.871      0.000         0.056     0.172
month_humidity           -0.0484      0.052     -0.928      0.354        -0.151     0.054
month_cloud_cover        -0.0483      0.021     -2.316      0.021        -0.089    -0.007
month_pressure            0.0614      0.023      2.613      0.009         0.015     0.107
month_maxtemp             0.2721      0.088      3.089      0.002         0.099     0.445
month_mintemp            -0.0652      0.094     -0.694      0.488        -0.250     0.119
month3_ddays50           -0.4772      0.204     -2.335      0.020        -0.878    -0.077
month3_ddays50_97        -0.0988      0.096     -1.026      0.305        -0.288     0.090
month3_ddays97           -0.0797      0.059     -1.350      0.177        -0.196     0.036
month3_humidity           0.0737      0.065      1.134      0.257        -0.054     0.201
month3_cloud_cover        0.0605      0.023      2.620      0.009         0.015     0.106
month3_pressure        